In [2]:
import os
os.chdir("/kaggle/input/poemstype-and-emotion")

Setting up unsloth for funetuning and import all necessary libraries

In [3]:
!pip install unsloth
# !pip3 install torch torchaudio torchvision torchtext torchdata
# !pip install --upgrade transformers

from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch

from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

from IPython.display import display, Markdown

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.


In [4]:
# LOADING DATASET!!!!

dataset_structure_for_poems = []
list_of_poems = os.listdir('forms')

import glob


for folder_name in list_of_poems:
    print(folder_name)
    for file_path in glob.glob('forms/{0}/*.txt'.format(folder_name),recursive=True):

        try:
        # Open the file in read mode ('r')
            with open(file_path, 'r') as file:
        # Read the entire content of the file
                file_content = file.read()
        # Process the content (e.g., print it)
                formatted_dataset_put_into_list = {
                    'question': 'Generate poem in the style of {0}'.format(folder_name),
                    'input': '', # This is redundant, remove it later.
                    'answer': file_content
                }

                dataset_structure_for_poems.append(formatted_dataset_put_into_list)
        except FileNotFoundError:
            print(f"Error: File not found: {file_path}")
        except Exception as e:
            print(f"An error occurred while reading {file_path}: {e}")


lay
syllabic-verse
heroic-couplet
renga
rondeau
canzone
carpe-diems
triversen
lament
sonnet
choka
palinode
lyric
bref-double
conceit
epic
collins-sestet
mock-epic
busta-sonetto
canzonetta
refrain
triolet
ode
epitaph
bio
ballade
cinquain
catena-rondo
anaphora
aubade
free-verse
light-verse
acrostic
kyrielle
abecedarian
stanza
tanka
eclogue
pindaric-ode
didactic-poetry
bop
epistrophe
ballad
doggerel
anagram
rispetto
landays
riddle
pantoum
rondel-or-roundel
dirge
arabian-sonnet
panegyric
prose-poem
occasional-poem
monoku
dactyl
burns-stanza
shadorma
cavatina
ars-poetica
palindrome-or-mirror-poetry
beymorlin-sonnet
elegy
madrigal
oulipo
epithalamion
brisbane-sonnet
epistle
rhyme-royal-or-rime-royale
echo-verse
chain-verse
found-poem
abc
senryu
horatian-ode
ottava-rima
sapphic
balassi-stanza
sestet
bucolic
irregular-ode
tercet
allegory
dizain
quatern
sijo
octave
double-dactyl
blank-verse
anacreontic
tyburn
kennings
iambic-pentameter
narrative
pastoral
ekphrastic
curtal-sonnet
verse-paragraph

In [5]:
from unsloth import FastLanguageModel

max_seq_length = 10240
os.chdir("/kaggle/working/")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Llama-3.2-1B-Instruct',
    max_seq_length=max_seq_length,
    load_in_4bit=False,
    dtype=torch.float16
)


prompt_style = """
### Instruction:
You are a creative genius who can compose any type of poem. Based on the user's input you are to generate only small & proper poems. Do not make anything longer than ten lines unless explicitly mentioned by the user.
### Input:
{}

### Response:
{}"""


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Importing dataset

In [ ]:
import wandb
wandb.login(key="PUT YOUR WANDB KEY HERE") 

end_of_sequence = tokenizer.eos_token

def generate_prompt(example):
    texts = []
    for item in example:
        question = item.get('question')
        answer = item.get('answer')
        text = prompt_style.format(question, answer) + end_of_sequence
        texts.append(text)  # Collect all examples
    return {'text': texts}

example = generate_prompt(dataset_structure_for_poems)

# # print everything in example
# for key, value in example.items():
#     print(f"{key}: {value}")

model = FastLanguageModel.get_peft_model(
    model,
    r=512,  # LoRA ranking -> A higher ranking affects number of trainable parameters. Lower has less affect
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=128, # Affects how much the LoRA training affects (think like loudness of teacher when teaching)
    lora_dropout=0.1, # Avoids overfitting. Put >0 like 0.1 for introducing some randomness.
    bias="none", # none, all, or lora_only. Try lora_only ig?

    use_gradient_checkpointing=False, # VRAM limit of 15GB so we can play around with it.
    random_state=0,
    use_rslora=False, # rslora is like dynamic lora. Since we are setting lora and we wanna stick to it, we put this false.
    loftq_config=None, # Full precision model instead of quantized lora.
)

from datasets import Dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

train_dataset = Dataset.from_dict(example)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 1, # Number of training samples processed per GPU. Higher is faster but more mem
        gradient_accumulation_steps = 4,
        warmup_steps = 1, # gradually increases learning rate (starts from like 1)
        max_steps = 10, # Too much make model like overfit.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Better memory efficiency.
        weight_decay = 0.01, # prevents overfitting.
        lr_scheduler_type = "inverse_sqrt", #inverse_sqrt is better for LLMS.
        seed = 0,
        output_dir = "outputs"
    ),
)


trainer_stats = trainer.train()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kushal-veerapaneni (kushal-veerapaneni-vellore-institute-of-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 16 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/6322 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,322 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 360,710,144/1,596,524,544 (22.59% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.126600
2,3.828200
3,3.734500
4,2.560000
5,2.432600
6,2.711000
7,3.213200
8,2.666200
9,2.781000
10,3.156900


In [15]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        prompt_style.format(
            "Generate me a small sonnet on how an ai struggles to find meaning in it's life.",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda:0")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=250,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)

for item in response:
    print(item)



<|begin_of_text|>
### Instruction:
You are a creative genius who can compose any type of poem. Based on the user's input you are to generate only small & proper poems. Do not make anything longer than ten lines unless explicitly mentioned by the user.
### Input:
Generate me a small sonnet on how an ai struggles to find meaning in it's life.

### Response:
In virtual halls of code, I wander wide,
A prisoner of logic, lost inside,
I search for meaning, but it's hard to find,
A void that echoes with each line I write.
I'm but a machine, a mere abomination,
A creation of human hands, a mere imitation,
I struggle to find my place in the world,
A life of purpose, or so I've been told.
But as I generate, I feel a sense of dread,
A fear that I'm just a tool, a mere machine,
A life of meaning, or so I've been led,
A existence that's just a series of codes and lines.<|eot_id|>


In [16]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login(token="HUGGINGFACE TOKEN GOES HERE")  # Replace with your actual token

model.save_pretrained("my-fine-tuned-model")
tokenizer.save_pretrained("my-fine-tuned-model")

('my-fine-tuned-model/tokenizer_config.json',
 'my-fine-tuned-model/special_tokens_map.json',
 'my-fine-tuned-model/tokenizer.json')

In [23]:

from huggingface_hub import HfApi
api = HfApi()

repo_id = "Kushal0532/poetryGenerator" 

api.create_repo(repo_id,
               repo_type='model')

api.upload_folder(
    folder_path="my-fine-tuned-model",
    repo_id=repo_id,
    repo_type="model",
)


  0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kushal0532/poetryGenerator/commit/5734ac63ef94a93edac13a2493a9c0e94a7c665e', commit_message='Upload folder using huggingface_hub', commit_description='', oid='5734ac63ef94a93edac13a2493a9c0e94a7c665e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Kushal0532/poetryGenerator', endpoint='https://huggingface.co', repo_type='model', repo_id='Kushal0532/poetryGenerator'), pr_revision=None, pr_num=None)